<a href="https://colab.research.google.com/github/theAkhileshRai/DeepLearning/blob/master/AssetLvl_Pred_lNN_BinaryClassifierDCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder    


from sklearn.metrics import confusion_matrix, classification_report

from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/Train_data_laest.csv',index_col=False)

In [ ]:
# df1 = pd.read_csv('/content/old_normalised_assets-2.csv',index_col=False)
# df2 = pd.read_csv('/content/new_normalised_assets-2.csv',index_col=False)

In [ ]:
df2 = df.loc[(df.Asset == 'Ashland_Unit256') | (df.Asset == 'Center station_C2') | (df.Asset == 'El Dorado_Unit J2502') | (df.Asset == 'Saltlake_K71010B')]

In [ ]:
# 'West_Turkey_Unit2', 'Cross bar_Unit1',
#        'Reagan_1_Unit6', 'Reagan_1_Unit3', 'Lightning_Unit2',
#        'Hobit_Unit2', 'Katy_Unit1116', 'Katy_Unit1117', 'HobitLP_Unit102',
#        'Goliad_Unit1100', 'Lightning_Unit4'

In [ ]:
df1 = df.loc[(df.Asset == 'West_Turkey_Unit2') | (df.Asset == 'Cross bar_Unit1') | (df.Asset == 'Reagan_1_Unit6') | (df.Asset == 'Reagan_1_Unit3')|
       (df.Asset == 'Lightning_Unit2')|(df.Asset == 'Hobit_Unit2')|(df.Asset == 'Katy_Unit1116')|(df.Asset == 'Katy_Unit1117')|
       (df.Asset == 'HobitLP_Unit102')|(df.Asset == 'Goliad_Unit1100')|(df.Asset == 'Lightning_Unit4')]

In [ ]:
df1.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,multi_class,Binary,stage
16265,01-01-2019 00:30,1,61.986816,71.426983,125.652596,136.611843,1152.29500,61.986816,159.149519,121.483613,317.142421,1152.29500,West_Turkey_Unit2,Normal,0,1.0
16266,01-01-2019 01:30,1,63.729848,74.751558,120.175697,132.502807,1152.51750,63.729848,162.484929,115.979466,306.485629,1152.51750,West_Turkey_Unit2,Normal,0,1.0
16267,01-01-2019 02:30,1,63.071994,74.110940,105.488888,116.486283,1152.13125,63.071994,167.057610,98.548758,273.648759,1152.13125,West_Turkey_Unit2,Normal,0,1.0
16268,01-01-2019 03:30,1,59.612113,69.365860,107.897634,117.815998,1151.63125,59.612113,161.399138,101.750428,279.490785,1151.63125,West_Turkey_Unit2,Normal,0,1.0
16269,01-01-2019 07:30,1,55.314189,66.512616,112.050268,123.957754,1152.40875,55.314189,153.976086,106.750482,283.539875,1152.40875,West_Turkey_Unit2,Normal,0,1.0


In [ ]:
df1.columns

Index(['time', 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean', 'Asset',
       'multi_class', 'Binary', 'stage'],
      dtype='object')

In [ ]:
df1['Binary'] = df1['Binary'].fillna(0)
df1['multi_class'] = df1['multi_class'].fillna(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df1.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,multi_class,Binary,stage
16265,01-01-2019 00:30,1,61.986816,71.426983,125.652596,136.611843,1152.29500,61.986816,159.149519,121.483613,317.142421,1152.29500,West_Turkey_Unit2,Normal,0,1.0
16266,01-01-2019 01:30,1,63.729848,74.751558,120.175697,132.502807,1152.51750,63.729848,162.484929,115.979466,306.485629,1152.51750,West_Turkey_Unit2,Normal,0,1.0
16267,01-01-2019 02:30,1,63.071994,74.110940,105.488888,116.486283,1152.13125,63.071994,167.057610,98.548758,273.648759,1152.13125,West_Turkey_Unit2,Normal,0,1.0
16268,01-01-2019 03:30,1,59.612113,69.365860,107.897634,117.815998,1151.63125,59.612113,161.399138,101.750428,279.490785,1151.63125,West_Turkey_Unit2,Normal,0,1.0
16269,01-01-2019 07:30,1,55.314189,66.512616,112.050268,123.957754,1152.40875,55.314189,153.976086,106.750482,283.539875,1152.40875,West_Turkey_Unit2,Normal,0,1.0


In [ ]:
df1.shape

(88194, 16)

In [ ]:
df1.columns

Index(['time', 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean', 'Asset',
       'multi_class', 'Binary', 'stage'],
      dtype='object')

In [ ]:
X_columns = [ 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',\
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',\
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean']

In [ ]:
len(X_columns)

11

In [ ]:
X = df1[X_columns]

In [ ]:
X.shape

(88194, 11)

# build  with mean

In [ ]:
X

,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean
16265,1,61.986816,71.426983,125.652596,136.611843,1152.29500,61.986816,159.149519,121.483613,317.142421,1152.295000
16266,1,63.729848,74.751558,120.175697,132.502807,1152.51750,63.729848,162.484929,115.979466,306.485629,1152.517500
16267,1,63.071994,74.110940,105.488888,116.486283,1152.13125,63.071994,167.057610,98.548758,273.648759,1152.131250
16268,1,59.612113,69.365860,107.897634,117.815998,1151.63125,59.612113,161.399138,101.750428,279.490785,1151.631250
16269,1,55.314189,66.512616,112.050268,123.957754,1152.40875,55.314189,153.976086,106.750482,283.539875,1152.408750
...,...,...,...,...,...,...,...,...,...,...,...
104454,2,57.404187,192.870092,45.099492,173.448757,1097.04375,57.404187,194.540939,45.360622,176.846614,1095.845625
104455,2,58.232545,192.783737,44.219374,169.162607,1094.24000,58.232545,195.095136,44.179864,172.503697,1094.654375
104456,2,58.345895,193.720827,43.146701,166.990013,1096.65000,58.345895,195.398975,43.390119,170.251174,1096.146250
104457,2,57.941348,194.445306,42.829713,167.715724,1095.04250,57.941348,196.355408,42.965954,170.994143,1093.078125


In [ ]:
y_true = df1.iloc[:,14]

In [ ]:
y_true.unique()

array([0, 1])

In [ ]:
#seed = 42
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_size)

In [ ]:
# X_val, X_test, y_val, y_test = train_test_split(X_Test, y_Test, test_size=0.75)

In [ ]:
from collections import Counter

In [ ]:
print(X_train.shape,y_train.shape)

(74964, 11) (74964,)


In [ ]:
len(X_test)

13230

In [ ]:
#len(X_val)

In [ ]:
Counter(y_train)

Counter({0: 54797, 1: 20167})

64->16->4->1

In [ ]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(11, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_3 = nn.Linear(64, 64)
        self.layer_4 = nn.Linear(64, 10)
        self.layer_5 = nn.Linear(10, 4)
        self.layer_out = nn.Linear(4, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.batchnorm4 = nn.BatchNorm1d(10)

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_test

array([[-0.2006034 , -0.57636651, -0.21016634, ...,  0.19874092,
         0.56126683, -0.08470778],
       [-0.2006034 , -0.32032674, -0.10114577, ...,  0.20308906,
         0.63328705, -0.08539346],
       [ 0.74320278,  1.08453403,  0.64179081, ..., -0.98921074,
        -1.00865184, -0.08506895],
       ...,
       [ 0.74320278, -1.26713144, -0.37686508, ...,  0.22063777,
         0.54226295, -0.13244165],
       [ 1.68700895, -2.33964649, -0.69361377, ...,  0.2299126 ,
         0.49293515, -0.21361567],
       [-1.14440957,  0.70781325, -0.78323681, ...,  1.08211572,
         0.3101377 , -0.25970382]])

In [ ]:
EPOCHS = 100
BATCH_SIZE = 75
LEARNING_RATE = 0.079

In [ ]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

y_train = y_train.values
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
# ## val data    
# class valData(Dataset):
    
#     def __init__(self, X_data):
#         self.X_data = X_data
        
#     def __getitem__(self, index):
#         return self.X_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)

test_data = testData(torch.FloatTensor(X_test))
# val_data = valData(torch.FloatTensor(X_val))

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(dataset=val_data, batch_size=1)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
model = binaryClassification()
model.to(device)

binaryClassification(
  (layer_1): Linear(in_features=11, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        #X_batch = X_batch
  
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

In [ ]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
confusion_matrix(y_test, y_pred_list)

array([[8732,  885],
       [ 273, 3340]])

# Old Assets

In [ ]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94      9617
           1       0.79      0.92      0.85      3613

    accuracy                           0.91     13230
   macro avg       0.88      0.92      0.90     13230
weighted avg       0.92      0.91      0.91     13230



In [ ]:
import pickle
filename = 'binarynn_dcp_model.pkl'

In [ ]:
torch.save(model.state_dict(),filename)


In [ ]:
model.load_state_dict(torch.load(filename))

<All keys matched successfully>

In [ ]:
model.eval()

binaryClassification(
  (layer_1): Linear(in_features=11, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

# Test Pred New Assets

In [ ]:
X = df2[X_columns]

In [ ]:
y_true = df2.iloc[:,14]

In [ ]:
X_test = scaler.fit_transform(X)

In [ ]:
test_data = testData(torch.FloatTensor(X_test))

In [ ]:
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [ ]:
X_test.shape

(16265, 11)

In [ ]:
y_pred_list = []
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())



# New Assets

In [ ]:
predictions=[]
predictions.append([y_pred_list[i][0][0] for i in range(len(y_pred_list))])

In [ ]:
print(classification_report(y_true, predictions[0]))

              precision    recall  f1-score   support

           0       0.47      0.95      0.63      3779
           1       0.98      0.67      0.80     12486

    accuracy                           0.74     16265
   macro avg       0.72      0.81      0.71     16265
weighted avg       0.86      0.74      0.76     16265

